In [10]:
%pip install -r requirements.txt

  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.65-py3-none-any.whl.metadata (5.8 kB)
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached langchain_google_genai-2.0.10-py3-none-any.whl.metadata (3.6 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.3.45-py3-none-any.whl.metadata (15 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached google_api_core-2.25.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_api_python_client-2.174.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### IMPORTS

In [ ]:
from os import getenv
from io import BytesIO
from os.path import exists
from re import search
from pandas import read_csv, read_sql
import sqlalchemy as sqlalc
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

class SemArquivoCabecalho(Exception):
    pass

class SemArquivoItens(Exception):
    pass

class SemArquivoZip(Exception):
    pass

class SemResposta(Exception):
    pass

#### <b>AGENTE 1: Aquisição de Documentos</b>
<b>Responsabilidade:</b> Obter e pré-processar documentos fiscais<br/><br/>
<b>Funcionalidades:</b>
<ul><li>Interface para upload manual de arquivos (PDF, imagens)</li></ul>
<ul><li>Integração com APIs de órgãos governamentais (SEFAZ)</li></ul>
<ul><li>Validação inicial de formato e integridade dos documentos</li></ul>
<ul><li>Organização e catalogação dos arquivos recebidos (Armazenando em banco de dados, se os arquivos responderem a pergunta)</li></ul>

In [ ]:
def agente1(pergunta,engine, arquivos,llm):

    print('\nExecutando agente 1...')

    # VALIDAÇÃO DE INTEGRIDADE -> IMPLEMENTAR PARA DETERMINAR SE O ARQUIVO REALMENTE É UM TIPO ZIP. NÃO FAZER PELA EXTENSÃO
    # FAZER

    """
        Utilizando a LLM para identificar se os campos e registros da base de documentos, sãa capazes de responder a pergunta
        do usuário.

        Se sim, os arquivos são persistidos no banco de dados, caso contrário, o arquivo é descartado.
    """
    # FORMATANDO A SAÍDA DA LLM COM JsonOutputParser
    class Resposta(BaseModel):
        resposta: str = Field(description="Responda Sim ou Não")

    parseador = JsonOutputParser(pydantic_object=Resposta)

    # CRIANDO O PROMPT PARA A LLM COM A SAIDA FORMATADA
    template = """É possível responder a pergunta {pergunta} do usuário baseado no dataframe {df} ? {resposta}"""

    prompt_template = PromptTemplate(
                                        template=template,
                                        input_variables=["pergunta","df"],
                                        partial_variables={"resposta" : parseador.get_format_instructions()}
                                    )

    # CRIANDO A CADEIA DE EXECUÇÃO PARA A LLM
    chain = prompt_template | llm | parseador

    # CATALOGANDO OS ARQUIVOS NO BD
    j=0

    inspector = sqlalc.inspect(engine) # INSPECTOR PARA LISTAR AS TABELAS DO BANCO DE DADOS

    for f in arquivos:

        # SERÁ CRIADO UM DATAFRAME PARA CADA ARQUIVO
        if f.get('cabecalho') is not None:
            dfcabecalho = read_csv(f.get('cabecalho'))

            # INSERINDO COLUNA COM O NOME DO ARQUIVO NO DATAFRAME
            dfcabecalho['ARQUIVO'] = f.get('nome_arquivo')
            df = dfcabecalho

            #print('Dataframe de cabeçalho: ',df)

            # INVOCANDO A LLM
            resposta = chain.invoke(input={"pergunta":pergunta, "df": df})['resposta']

            if resposta == 'Sim':
                j+=1

                print('Sim para o arquivo: ',f.get('nome_arquivo'))

                # PRECISA VERIFICAR SE A TABELA JÁ EXISTE NO BANCO DE DADOS ANTES DE LER
                if 'NFCABECALHO' in inspector.get_table_names():
                    dftable = read_sql('NFCABECALHO', con=engine)

                    #print('dftable NFCABECALHO\n',dftable)

                    # CUIDANDO DE DUPLICIDADE
                    df = df[~df['CHAVE DE ACESSO'].isin(dftable['CHAVE DE ACESSO'])]

                # INSERINDO NO BANCO DE DADOS
                df.to_sql(name='NFCABECALHO', con=engine, if_exists='append', index=False)

                continue

            else:
                continue

        if f.get('itens') is not None:
            dfitens = read_csv(f.get('itens'))

            # INSERINDO COLUNA COM O NOME DO ARQUIVO NO DATAFRAME
            dfitens['ARQUIVO'] = f.get('nome_arquivo')
            df = dfitens

            #print('Dataframe de itens: ',df)

            # INVOCANDO A LLM
            resposta = chain.invoke(input={"pergunta":pergunta, "df": df})['resposta']

            if resposta == 'Sim':
                j+=1

                print('Sim para o arquivo: ',f.get('nome_arquivo'))

                 # PRECISA VERIFICAR SE A TABELA JÁ EXISTE NO BANCO DE DADOS ANTES DE LER
                if 'NFITENS' in inspector.get_table_names():
                    dftable = read_sql('NFITENS', con=engine)

                    #print('dftable NFINTENS\n',dftable)

                    # CUIDANDO DE DUPLICIDADE
                    df = df[~df['CHAVE DE ACESSO'].isin(dftable['CHAVE DE ACESSO'])]

                # INSERINDO NO BANCO DE DADOS
                df.to_sql(name='NFITENS', con=engine, if_exists='append', index=False)

                continue

            else:
                continue

    if j == 0:
        return "Não"

    else:
        return "Sim"

#### <b>AGENTE 2: Extração e Aprendizado</b>
<b>Responsabilidade:</b> Processar documentos e extrair dados relevantes<br/><br/>
<b>Funcionalidades:</b>
<ul><li>OCR avançado para digitalização de documentos</li></ul>
<ul><li>NLP para identificação e extração de campos específicos</li></ul>
<ul><li>IA para adaptação a novos layouts</li></ul>
<ul><li>Validação cruzada de dados extraídos</li></ul>

In [ ]:
def agente2(pergunta,llm,engine):

    print('\nExecutando agente 2...')

    # FORMATANDO A SAÍDA DA LLM COM JsonOutputParser
    class Query(BaseModel):
        query: str = Field(description='Esta é a query com DISTINCT e o nome de cada coluna entre "')

    parseador = JsonOutputParser(pydantic_object=Query)

    # CRIANDO O PROMPT PARA A LLM COM A SAIDA FORMATADA
    template_query = """Qual query deve ser executada na tabela NFCABECALHO com as colunas {colunas_tab_cabecalho} ou tabela NFITENS com as colunas {colunas_tab_itens} para responder
    a pergunta {pergunta}? Se a query envolver as duas tabelas, deve ser feito um JOIN entre elas utlizando a coluna "CHAVE DE ACESSO" como chave. {formatacao_saida}"""

    prompt_template_query = PromptTemplate(
                                            template=template_query,
                                            input_variables=["pergunta","colunas_tab_cabecalho","colunas_tab_itens"],
                                            partial_variables={"formatacao_saida" : parseador.get_format_instructions()}
                                          )

    # CRIANDO A CADEIA DE EXECUÇÃO PARA A LLM
    chain = prompt_template_query | llm | parseador

    with engine.connect() as con:
        query1 = sqlalc.text('PRAGMA table_info(NFCABECALHO)')
        rs = con.execute(query1)
        rows = rs.fetchall()
        colunas_query1 = sorted([col[1] for col in rows])
        #print('Colunas query1: ', colunas_query1)

        query2 = sqlalc.text('PRAGMA table_info(NFITENS)')
        rs = con.execute(query2)
        rows = rs.fetchall()
        colunas_query2 = sorted([col[1] for col in rows])
        #print('Colunas query2: ', colunas_query2)


    query = chain.invoke(input={"pergunta":pergunta, "colunas_tab_cabecalho":colunas_query1,"colunas_tab_itens":colunas_query2})['query']

    print('\nQuery: ',query)

    resposta = query

    return resposta

#### <b>AGENTE 3: Resposta e Interação</b>
<b>Responsabilidade:</b> Interface inteligente com usuários<br/><br/>
<b>Funcionalidades:</b>
<ul><li>Integração com LLMs para consultas em linguagem natural.</li></ul>

In [ ]:
def agente3(pergunta,arquivos):

    if not exists('nfs_data.db'): # CRIAÇÃO DO BANCO DE DADOS PARA A PRIMEIRA EXECUÇÃO
        print('\nCriando o banco de dados nfs_data...')
        DATABASE_URL = "sqlite:///nfs_data.db" # Define o nome do arquivo do banco de dados
        engine = sqlalc.create_engine(DATABASE_URL)

    else:
        engine = sqlalc.create_engine("sqlite:///nfs_data.db") # Conecta ao banco de dados existente


    # INTEGRAÇÃO COM A LLM
    load_dotenv() # CARREGANDO O ARQUIVO COM A API_KEY

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",  # ou "gemini-2.0-pro"
        temperature=0.5,
        google_api_key=getenv("GOOGLE_API_KEY")
    )


    try:
            print('\nExecutando agente 3...')

            print('\nPergunta: ',pergunta)

            resposta = agente1(pergunta,engine,arquivos,llm) # A ENGINE NÃO É FECHADA AUTOMATICAMENTE, APENAS AS CONEXÕES QUANDO USADAS COM WITH

            if resposta == "Sim":
                query = agente2(pergunta,llm,engine)

                # # OBTENÇÃO DO RESULTADO DA QUERY
                with engine.connect() as con:
                        df = read_sql(query, con)
                        resposta = df

            elif resposta == "Não":
                    raise SemResposta

            # elif resposta == "SemArquivoZip":
            #     raise SemArquivoZip

            elif resposta == "SemArquivoCabecalho":
                    raise SemArquivoCabecalho

            elif resposta == "SemArquivoItens":
                    raise SemArquivoItens


    # EXECUÇÃO DAS EXCEÇÕES
    except SemArquivoCabecalho:
            resposta = "SemArquivoCabecalho"
            print('\nResposta: ', resposta)

    except SemArquivoItens:
            resposta = "SemArquivoItens"

    # except SemArquivoZip:
    #     caminho_absoluto = abspath(diretorio)
    #     print(f'\nNão há arquivos zipados no diretório {caminho_absoluto}!\n')

    except SemResposta:
            resposta = "SemResposta"

    print('\nResposta\n',f'{resposta}')
    
    return resposta

#### <b>TESTANDO</b>

In [14]:
if __name__ == "__main__":

     #arquivo = ".\\202401_NFS - new.zip"  # Diretório onde os arquivos zipados estão localizados
     
     arquivo = "202401_NFS.zip"  # Diretório onde os arquivos zipados estão localizados
     
#    # EXEMPLOS DE PERGUNTA PARA TESTE. ELAS DEVEM SER OBTIDAS DO FRONTEND
     pergunta = "Qual é a chave de acesso da nota 3510129 ?"
     pergunta = "Quem descobriu o Brasil ?"
     pergunta = "Qual é a descrição dos serviços de nf com número 2525 ?"
     pergunta = "Qual é a descrição dos serviços e a natureza da operação da nf com número 2525 ?"

     resposta = agente3(pergunta, arquivo)  # Chama a função principal com a pergunta e o diretório
     print('\nResposta: \n',resposta)



Executando agente 3...

Pergunta:  Qual é a descrição dos serviços e a natureza da operação da nf com número 2525 ?

Executando agente 1...
Arquivos descompactados: ['202401_NFs_Cabecalho.csv', '202401_NFs_Itens.csv']
Sim para o arquivo:  202401_NFs_Cabecalho.csv
Sim para o arquivo:  202401_NFs_Itens.csv

Executando agente 2...

Query:  SELECT DISTINCT "DESCRIÇÃO DO PRODUTO/SERVIÇO", "NATUREZA DA OPERAÇÃO" FROM NFITENS WHERE "NÚMERO" = '2525'

Resposta: 
    DESCRIÇÃO DO PRODUTO/SERVIÇO                NATUREZA DA OPERAÇÃO
0  LANTERNA TATERAL CARRETA LED  VENDA DE MERCADORIA FORA DO ESTADO
1            CINEMATICO RODO-AR  VENDA DE MERCADORIA FORA DO ESTADO
2  ESPIRAL NYLON CABINE AMARELO  VENDA DE MERCADORIA FORA DO ESTADO
